<img src='Abnb_logo.jpg'>

In [29]:
import requests
from bs4 import BeautifulSoup
import time

In [30]:
url = "https://www.airbnb.com.pe/s/Paris/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&query=Par%C3%ADs%2C%20Francia&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&date_picker_type=calendar&checkin=2023-04-19&checkout=2023-04-22&adults=1&source=structured_search_input_header&search_type=autocomplete_click"

r = requests.get(url)

r.status_code

s = BeautifulSoup(r.text,'lxml')


from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

In [31]:
options = webdriver.ChromeOptions()
options.add_argument("--incognito")

driver = webdriver.Chrome(executable_path=r"F:\\Chrome driver\\chromedriver.exe", options = options)

driver.get(url)

<ipython-input-31-1fd30b10088d>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"F:\\Chrome driver\\chromedriver.exe", options = options)


### OJO:
##### Esta def obtener_info_principal(). No serviría apra el dataframe ya que son listas y no cada lista una col.
##### al menos que las dividas en varias listas y hagas el append asi

In [32]:
titles = []
precios = []
precios_noche = []
camas = []
hallazgos = []

rates = []
cantidad_rates_airbnb = []

def obtener_info_principal():
    ### Este es el bloque dond se sencuentra tiulo, precio, camas rate etc
    bloques = driver.find_elements(By.XPATH, "//div[@class='g1qv1ctd cb4nyux dir dir-ltr']")
    bloques = bloques[:4]

    ### Este es e bloque dodne se enecuntra HALLAGOS
    bloques_hallazgo = driver.find_elements(By.XPATH, "//div[@class='c14dgvke dir dir-ltr']")
    bloques_hallazgo = bloques_hallazgo[:4]


    ## para bloque en bloquese encontramos titulo, rpecio, etc
    for x in bloques:   
        time.sleep(1)
        precio_total = x.find_element(By.XPATH, ".//div[@class='_tt122m']")
        precio_total = precio_total.find_element(By.XPATH, ".//span[@aria-hidden='true']")
        precio_total = precio_total.text

        precio_total = precio_total.split()
        vaina_1 = precio_total[0]
        vaina_2 = vaina_1[2:]

        # Tienes que eliminar las comas de los precios 

        coma=[",","S","$","U","/"]
        res=[j for i,j in enumerate(vaina_2) if j not in coma]
        price = "".join(res)
        price = int(price)

        time.sleep(1.6)



                ### precio x noche, ya que este a veces no se encuentra usamos un TRY
        try: 
            precio_noche = x.find_element(By.XPATH, ".//span[@class='_tyxjp1']")
            time.sleep(0.6)
            precio_noche = precio_noche.text
            precio_noche = precio_noche.split()
            vaina_1_n = precio_noche[0]
        except:
            precio_noche = x.find_element(By.XPATH, ".//span[@class='_1y74zjx']")
            time.sleep(0.6)
            precio_noche = precio_noche.text
            precio_noche = precio_noche.split()
            vaina_1_n = precio_noche[0]

        #### OJO todo depende de la moneda en la que esté la vaina esta, si está en euro o sol peude que sea diferente
        vaina_2_n = vaina_1_n[2:]

        # Tienes que eliminar las comas de los precios 

        coma=[",","S","$","U","/"]
        res=[j for i,j in enumerate(vaina_2_n) if j not in coma]
        price_n = "".join(res)
        price_n = int(price_n)

        time.sleep(1)

        cama = x.find_element(By.XPATH, ".//div[@class='f15liw5s s1cjsi4j dir dir-ltr'][2]")
        cama = cama.find_element(By.XPATH, ".//span[@class=' dir dir-ltr']")
        cama = cama.text
        cama = cama.split()
        cama = cama[0]
        time.sleep(1)

        title = x.find_element(By.XPATH, ".//div[@data-testid='listing-card-title']")
        title = title.text
        time.sleep(1)

        ## como algunos son neuvos no tienen rates
        try: 
            rate = x.find_element(By.XPATH, ".//span[@class='r1dxllyb dir dir-ltr']")
            rate = rate.text
            rate = rate.split()
            rate_1 = rate[0]
            rate_1 = float(rate_1)
            cantidad_rates = rate[1]

            coma=[",","(",")","U","/"]
            res=[j for i,j in enumerate(cantidad_rates) if j not in coma]
            cantidad_rates = "".join(res)
            cantidad_rates = int(cantidad_rates)

            cantidad_rates_airbnb.append(cantidad_rates)
            time.sleep(0.6)
            rates.append(rate_1)
            time.sleep(0.6)

        except:
            cantidad_rates_airbnb.append("Nuevo")
            time.sleep(0.6)
            rates.append("Nuevo")
            time.sleep(0.6)

        titles.append(title)
        time.sleep(0.6)

        precios.append(price)        
        time.sleep(0.6)

        precios_noche.append(price_n)        
        time.sleep(0.6)

        camas.append(cama)        
        time.sleep(0.6)


        ## como los hallazgos estna en otor loque uso ese bloque
    for z in bloques_hallazgo:
        time.sleep(0.6)
        try:
            time.sleep(0.6)
            hallazgo_a = z.find_element(By.XPATH, ".//div[@class='t1mwk1n0 dir dir-ltr']")
            time.sleep(0.7)
            hallazgo_a = hallazgo_a.text
            time.sleep(0.3)
            hallazgos.append(hallazgo_a)

        except: 
            hallazgos.append("null")

In [33]:
obtener_info_principal()

In [34]:
precios

[662, 638, 781, 711]

# Interactuar y mandar keys

In [35]:
def partes_llegas(lugar_partida):
    
    donde_partes = driver.find_element(By.XPATH, "//div[@class='f1xx50dm dir dir-ltr']")
    donde_partes.click()
    ## Le das a click en el boton donde dice "en cualqueir lguar del mundo" osea donde buscas el lugar donde vas a estar p
    
    time.sleep(0.7)
    #ivycze5 dir dir-ltr
    ### clcikeas en donde vas a escribir Just in case
    escribe_donde = driver.find_element(By.XPATH, "//input[@class='ivycze5 dir dir-ltr']")
    escribe_donde.click()
    
    lugars = lugar_partida
    
    ## Envias el nombre del lguar letra por letra
    for character in lugars:
        escribe_donde.send_keys(character)
        time.sleep(0.3)
    
    # clcikeas la primera opción, también puedes hacer un match del lugar exacto 
    clickea_el_primero = driver.find_element(By.XPATH, "//div[@class='_r1t6ga']")
    clickea_el_primero.click()


In [36]:
def repetir_mes_llegada(mes,dia):
    
    # clcik  en el button FECHA llegada a la ciudad o lugar
    # directamente se clickea, asi que aca no es necesario
    
    ## encotramos el boton par avanzar mes SIGUIENTE, este caos es el segundo, tambien est el arie-label  : Haz clic en la flecha de la derecha para cambiar al mes siguiente.
    bloque_button = driver.find_element(By.XPATH, "//div[@class='_qz9x4fc']")
    boton = bloque_button.find_element(By.XPATH, ".//button[@type='button']")
    ### mandamos el mes a LWOER
    mes = str(mes.lower())
    
    ## Cogemos los meses que aparecen en la webpage una vez hacemos click en fechas, en desktop son 2 mese sque aparecen 

    meses_llegada = driver.find_elements(By.XPATH, "//h2[@class='_14i3z6h']")
    time.sleep(1)
    meses_llegada = meses_llegada[1:3]
    meses_llegada_1 = []

    ## para cada mes en la lista de meses sacada arriba de la pagina, lo spliteamos, madnamos a lower y lo emtemos a una nueva lista
    for x in meses_llegada:
        texto = x.text
        time.sleep(1)
        texto = texto.split(" ")
        texto = texto[0].lower()
        meses_llegada_1.append(texto)
        time.sleep(1)

    ## en caso el mes enviado en la DEF está en la lista se le manda el día y se cliclea el día
    if mes in meses_llegada_1: 
        # el blqieu serían los calendarios de los 3 meses 
        bloque  = driver.find_elements(By.XPATH, "//table[@class='_cvkwaj']")
        time.sleep(1)
        # hay dos bloques extras fantamsas por lo tanto solo nos quedamos con los necesarios
        bloque = bloque[1:3]
        time.sleep(1)
        ## BUSCAS POR "NAME" EN EL INDEX, osea busco por MES, dodne se posiciona para hacer el match
        indice = meses_llegada_1.index(mes)
        time.sleep(1)
        bloque = bloque[indice]
        time.sleep(1)
        llegada = bloque.find_elements(By.XPATH, ".//td[@role='button']")
        x = dia
        x = x-1    # te coje día +1 entonce sque sea *-1   esto por q tabaja con lista p y el indice comienza desde el 0 webas
        time.sleep(1)
        ## haces un amthc de dia igual que antes
        fecha_que_quiero = llegada[x]
        fecha_que_quiero.click()
        
        # si el mes enviado en el DEF NO ESTA en la lista, se clickea el boton "SIGUEITNE MES" hasta encontrarlo
        
    else:
        boton.click()
        time.sleep(1)
        boton.click()
        time.sleep(1)
        repetir_mes(mes,dia)
    
    

In [50]:
 def repetir_mes_salida(mes,dia):
    
    # clcik  en el button FECHA llegada a la ciudad o lugar
    # directamente se clickea, asi que aca no es necesario
    

    ## encotramos el boton par avanzar mes SIGUIENTE, este caos es el segundo, tambien est el arie-label  : Haz clic en la flecha de la derecha para cambiar al mes siguiente.
    bloque_button = driver.find_element(By.XPATH, "//div[@class='_qz9x4fc']")
    boton = bloque_button.find_element(By.XPATH, ".//button[@type='button']")
    ### mandamos el mes a LWOER
    mes = str(mes.lower())
    
    ## Cogemos los meses que aparecen en la webpage una vez hacemos click en fechas, en desktop son 2 mese sque aparecen 

    meses_llegada = driver.find_elements(By.XPATH, "//h2[@class='_14i3z6h']")
    time.sleep(1)
    meses_llegada = meses_llegada[1:3]
    meses_llegada_1 = []

    ## para cada mes en la lista de meses sacada arriba de la pagina, lo spliteamos, madnamos a lower y lo emtemos a una nueva lista
    for x in meses_llegada:
        texto = x.text
        time.sleep(1)
        texto = texto.split(" ")
        texto = texto[0].lower()
        meses_llegada_1.append(texto)
        time.sleep(1)

    ## en caso el mes enviado en la DEF está en la lista se le manda el día y se cliclea el día
    if mes in meses_llegada_1: 
        # el blqieu serían los calendarios de los 3 meses 
        bloque  = driver.find_elements(By.XPATH, "//table[@class='_cvkwaj']")
        time.sleep(1)
        # hay dos loques extras fantamsas por lo tanto solo nos quedamos con los necesarios
        bloque = bloque[1:3]
        time.sleep(1)
        ## BUSCAS POR "NAME" EN EL INDEX, osea busco por MES, dodne se posiciona para hacer el match
        indice = meses_llegada_1.index(mes)
        time.sleep(1)
        bloque = bloque[indice]
        time.sleep(1)
        llegada = bloque.find_elements(By.XPATH, ".//td[@role='button']")
        x = dia
        x = x-1    # te coje día +1 entonce sque sea *-1   esto por q tabaja con lista p y el indice comienza desde el 0 webas
        time.sleep(1)
        ## haces un amthc de dia igual que antes
        fecha_que_quiero = llegada[x]
        fecha_que_quiero.click()
        
        # si el mes enviado en el DEF NO ESTA en la lista, se clickea el boton "SIGUEITNE MES" hasta encontrarlo
    else:
        boton.click()
        time.sleep(1)
        boton.click()
        time.sleep(1)
        repetir_mes_salida(mes,dia)
    

In [38]:
def cuantos_pajeros(adultos=None,ninos=None,bebes=None,perros=None):
    
    if adultos == 0:
        print("eres IDIOTA?")
        return
    
    click_cuantos = driver.find_element(By.XPATH, "//div[@class='p1kudodg dir dir-ltr']")
    click_cuantos.click()
    time.sleep(1)
           

    boomers = driver.find_element(By.XPATH, "//div[@id='stepper-adults']")
    boomers_boton = boomers.find_element(By.XPATH, ".//button[@class='_ul9u8c'][2]")
    time.sleep(1)
    
    n = 0
    if adultos is None or adultos == 1:
        pass
    else:
        while n < adultos:
                n += 1
                time.sleep(0.3)
                boomers_boton.click()
    
    
    mocosos = driver.find_element(By.XPATH, ".//div[@id='stepper-children']")
    mocosos_boton = mocosos.find_element(By.XPATH, ".//button[@class='_ul9u8c'][2]")
    time.sleep(1)
    n = 0
    if ninos == 0 or ninos == None:
        pass
    else:
        while n < ninos:
            n += 1
            time.sleep(0.4)
            mocosos_boton.click()

    time.sleep(1.7)
       
    wawas = driver.find_element(By.XPATH, ".//div[@id='stepper-infants']")
    wawas_boton = wawas.find_element(By.XPATH, ".//button[@class='_ul9u8c'][2]")
    time.sleep(1)
    n = 0
    if bebes == 0 or bebes == None:
        pass
    else:
        while n < bebes:
            n += 1
            time.sleep(0.6)
            wawas_boton.click()
            
    time.sleep(1.7)
 

    mascotas = driver.find_element(By.XPATH, ".//div[@id='stepper-pets']")
    mascotas_boton = mascotas.find_element(By.XPATH, ".//button[@class='_ul9u8c'][2]")
    time.sleep(1)
    n = 0
    if perros == 0 or perros == None:
        pass
    else:
        while n < perros:
            n += 1
            time.sleep(0.6)
            mascotas_boton.click()
            
    time.sleep(1.7)


In [39]:
def click_buscar():
    
    buscaras = driver.find_element(By.XPATH, "//span[@class='t1l18xbv dir dir-ltr']")
    buscaras.click()

## Acaban las interaciones y send kyes

In [61]:
partes_llegas("Madrid")

In [62]:
repetir_mes_llegada("abril",25)

In [63]:
repetir_mes_salida("may",29)

In [64]:
cuantos_pajeros(2,1,1,1)

In [65]:
click_buscar()
## IATAAAA IATAAAAA YOOOOOOOOOO

In [280]:
titles

['Apartamento en Clichy',
 'Habitación de hotel en París',
 'Apartamento en París',
 'Apartamento en París']

In [281]:
precios

[1715, 5499, 3561, 2512]

In [282]:
precios_noche

[1715, 5499, 3561, 2512]

In [283]:
camas

['2', '2', '2', '1']

In [284]:
hallazgos

['Un hallazgo excepcional', None, None, 'Un hallazgo excepcional']

In [285]:
rates

[['4.48', '(115)'], ['4.92', '(145)'], 'Nuevo', ['4.78', '(32)']]

In [286]:
cantidad_rates_airbnb

['(115)', '(145)', 'Nuevo', '(32)']

In [46]:
 driver.switch_to.window(driver.window_handles[0])

listas = hazlasasa sa s
for x in listas:
    for z in x:
        l = list(z)
    

## SACAR INFO DE CADA AIRBNB,
##### haciendo click en cada airbnb y scrapear sus vainas

In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [66]:
url = "https://www.airbnb.com.pe/s/Paris/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&query=Par%C3%ADs%2C%20Francia&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&date_picker_type=calendar&checkin=2023-04-19&checkout=2023-04-22&adults=1&source=structured_search_input_header&search_type=autocomplete_click"
#url= "https://www.airbnb.com.pe/rooms/710789383048240528?adults=1&check_in=2023-04-19&check_out=2023-04-22&federated_search_id=46fe1005-a6b6-4c5c-9f9d-8f4f2b66e20c&source_impression_id=p3_1676838689_a%2BpLM6Ws3w9IR4x5"
r = requests.get(url)

r.status_code

s = BeautifulSoup(r.text,'lxml')


from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

In [67]:
options = webdriver.ChromeOptions()
options.add_argument("--incognito")

driver = webdriver.Chrome(executable_path=r"F:\\Chrome driver\\chromedriver.exe", options = options)

driver.get(url)

<ipython-input-67-1fd30b10088d>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"F:\\Chrome driver\\chromedriver.exe", options = options)


### Before we scrap we need to create the CSV with the headers

In [68]:
import csv
header = ['date','titulo', 'huespedes','banos','verificado','habitaciones','camas','limpieza_rate','comunicacion_rate',
          'llegada_rate','fiabilidad_rate','ubicacion_rate','precio_rate','reviews_lista','que_suerte','aircover','idver',
          'superanfitrion','Promotor_abnb','name_anfitiron','fecha_registro',
          'zona_de_trabajo','acceso_sin_hr_restriccion','cancelacion_gratis','dscto_mensual',
          'dscto_semanal','tarifa_service','tarifa_cleaning','precio_noche_mes_week','precio_total',
          'precio_original','q_resenas','puntaje_place','idiomas','resposne_rate','response_time',
          'servicios_solos','bano_servicios','dormitorio_lavadero','entretenimiento','familia','calefaccion_refrigeracion',
          'seguridad_hogar','internet_oficina','utensilios_vajilla','estacionamiento_instalaciones','ubicacion_servicios',
          'exterior_servicios','servicios_servicios','no_incluidos','reglas_casa','seguridad_y_propiedad',
          'politica_cancelacion']

with open('abnb_SSS_scrapper.csv', 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [80]:
### ESTE ES EL GRANDAZO QUE QUIERO AHCER


    
def get_todo_de_c_abnb():   
    import datetime
    import csv

    ### Este es el bloque dond se sencuentra tiulo, precio, camas rate etc
    try: 
        bloques = driver.find_elements(By.XPATH, "//div[@class='g1qv1ctd cb4nyux dir dir-ltr']")
        bloques = bloques[5:7]
        time.sleep(0.5)

        ### Este es e bloque dodne se enecuntra HALLAGOS o la IMAGEN de cada abnb esa cosa. Ahi haremos click
        ### para poder entrar a cada ABNB y sacar toda la info posible
        bloques_hallazgo = driver.find_elements(By.XPATH, "//div[@class='c14dgvke dir dir-ltr']")
        
        bloques_hallazgo = bloques_hallazgo[5:7]
        time.sleep(0.5)
        

        for x in bloques_hallazgo:   
        
            #### declaramos las listas en las que se va a hacer el APPEND
            #### ---------------------------------------------------------------------------------------------
            
            global titulo_abnb

            global huespedes
            global banos
            global verificados 
            global habitacion 
            global cama 

            global limpieza_rate
            global comunicacion_rate 
            global llegada_rate
            global fiabilidad_rate 
            global ubicacion_rate 
            global precio_rate

            global reviews_lista

            global que_suerte

            global aircover 
            
            
            global idver
            global superanfitrion
            global Promotor_abnb 
            global name_anfitiron 
            global fecha_registro
            
            global zona_de_trabajo
            global acceso_sin_hr_restriccion
            global cancelacion_gratis 
            global dscto_mensual 
            global dscto_semanal
            
            global tarifa_service
            global tarifa_cleaning
            global precio_noche_mes_week 
            global precio_total 
            global precio_original
            
            global q_resenas 
            global puntaje_place

            global idiomas 
            global resposne_rate
            global response_time

            global servicios_solos 

            global bano_servicios
            global dormitorio_lavadero
            global entretenimiento

            global familia 
            global calefaccion_refrigeracion
            global seguridad_hogar 

            global internet_oficina 
            global utensilios_vajilla 
            global estacionamiento_instalaciones

            global ubicacion_servicios
            global exterior_servicios
            global servicios_servicios 
            global reglas_casa
            global seguridad_y_propiedad
            global politica_cancelacion
            
            ##### ARINONEKOOOOOOO ONO SHITUHEEEEEE
            
            titulo_abnb = []
            huespedes = []
            banos = []
            verificados = []
            habitacion = []
            cama = []
            limpieza_rate = []
            comunicacion_rate = []
            llegada_rate = []
            fiabilidad_rate = []
            ubicacion_rate = []
            precio_rate = []
            reviews_lista=[]
            que_suerte = []
            aircover = []       
            
            idver = []
            superanfitrion = []
            Promotor_abnb = []

            name_anfitiron = []
            fecha_registro = []


            zona_de_trabajo = []
            acceso_sin_hr_restriccion = []
            cancelacion_gratis = []

            dscto_mensual = []
            dscto_semanal = []
            tarifa_service = []
            tarifa_cleaning = []
            precio_noche_mes_week = []
            precio_total = []
            precio_original = []

            q_resenas = []
            puntaje_place = []

            idiomas = []
            resposne_rate = []
            response_time = []

            servicios_solos = []
            bano_servicios = []
            dormitorio_lavadero = []
            entretenimiento = []
            familia = []
            calefaccion_refrigeracion = []
            seguridad_hogar = []
            internet_oficina = []
            utensilios_vajilla = []
            estacionamiento_instalaciones = []
            ubicacion_servicios = []
            exterior_servicios = []
            servicios_servicios = []
            reglas_casa = []
            seguridad_y_propiedad = []
            politica_cancelacion = []
            
                        
            #### ---------------------------------------------------------------------------------------------
            
            
            time.sleep(10.6)
            x.click()
            time.sleep(1.8)

            print(driver.current_url)            
            time.sleep(0.6)

            driver.switch_to.window(driver.window_handles[1])
            time.sleep(2.2)
            
            print(driver.current_url)    
            
            time.sleep(1.5)

            try:
                cerrar_pop_up = driver.find_element(By.XPATH, "//div[@class='_1piuevz']")
                cerrar_pop_up.click()
                time.sleep(0.8)

            except:
                pass
            
            ## entonces para cada uno corremos las definicioes que scrapean las cosas de los ABNBs
            get_importantes_abnb()
            time.sleep(0.8)
            get_things_del_anfi()
            time.sleep(0.8)
            get_name_del_anfi()
            time.sleep(0.8)
            get_cosas_abnb()
            time.sleep(0.8)
            get_prices_dscts_abnb()
            time.sleep(0.8)
            communication_anfit()
            time.sleep(0.8)
            get_servicios_abnb()
            time.sleep(0.8)
            politicas_abnb()
            time.sleep(0.8)            
            
            
            date = datetime.date.today()
            
            data =     [date, titulo_abnb,huespedes, banos , verificados ,habitacion , cama , limpieza_rate , 
            comunicacion_rate , llegada_rate , fiabilidad_rate , ubicacion_rate , precio_rate , reviews_lista, 
            que_suerte , aircover , idver , superanfitrion , Promotor_abnb , name_anfitiron , 
            fecha_registro , zona_de_trabajo ,acceso_sin_hr_restriccion , cancelacion_gratis ,dscto_mensual ,
            dscto_semanal , tarifa_service ,tarifa_cleaning , precio_noche_mes_week , precio_total ,
            precio_original , q_resenas , puntaje_place , idiomas , resposne_rate , response_time ,            
            servicios_solos , bano_servicios , dormitorio_lavadero , entretenimiento , familia , calefaccion_refrigeracion ,
            seguridad_hogar , internet_oficina , utensilios_vajilla , estacionamiento_instalaciones , ubicacion_servicios ,
            exterior_servicios , servicios_servicios , no_incluidos , reglas_casa , seguridad_y_propiedad ,
            politica_cancelacion]            
            
            ### NTES DE AHCERLO REVISA SI ES SOBREESCRITO O RECIEN NUEVO
            with open('abnb_SSS_scrapper.csv', 'a+', newline='', encoding='UTF8') as f:
                writer = csv.writer(f)
                writer.writerow(data)
            
            time.sleep(3.8)
            
            ## regresamos a la pagina donde están todas las opciones de abnbs que tenemos
            driver.close()
            time.sleep(2)
            driver.switch_to.window(driver.window_handles[0])

    except:
        pass
    

In [81]:
get_todo_de_c_abnb()

https://www.airbnb.com.pe/s/Paris/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&query=Par%C3%ADs%2C%20Francia&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&date_picker_type=calendar&checkin=2023-04-19&checkout=2023-04-22&adults=1&source=structured_search_input_header&search_type=autocomplete_click
https://www.airbnb.com.pe/rooms/710789383048240528?adults=1&check_in=2023-04-19&check_out=2023-04-22&federated_search_id=c2b6188a-9d2b-4b10-b874-8fa7f3540640&source_impression_id=p3_1677982691_tcBw19ueRId9aXZV
https://www.airbnb.com.pe/s/Paris/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&query=Par%C3%ADs%2C%20Francia&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&date_picker_type=calendar&checkin=2023-04-19&checkout=2023-04-22&adults=1&source=structured_search_input_header&search_type=autocomplete_click
https://www.airb

In [111]:
driver.switch_to.window(driver.window_handles[0])

In [82]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
df1 = pd.read_csv("abnb_SSS_scrapper.csv")
df1
#df1[["titulo"]]
### PAREC QUE SIIIIIIUUUUUUUUU

date                                                          titulo  \
0  2023-03-04     ['habitación compartida en alojamiento - anfitrión: guoxi']   
1  2023-03-04  ['habitación privada en vivienda rentada - anfitrión: julien']   

       huespedes                       banos verificado         habitaciones  \
0  ['1 huésped']  [' · 3 baños compartidos']         []  [' · 1 habitación']   
1  ['1 huésped']    [' · 1 baño compartido']         []  [' · 1 habitación']   

           camas limpieza_rate comunicacion_rate llegada_rate fiabilidad_rate  \
0  [' · 1 cama']       ['4.9']           ['4.9']      ['4.9']         ['4.8']   
1  [' · 1 cama']       ['4.8']           ['4.9']      ['4.9']         ['4.9']   

  ubicacion_rate precio_rate  \
0        ['4.7']     ['4.9']   
1        ['4.5']     ['4.9']   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [ ]:
### verificado no sirve

In [21]:
########################################uiasuihaiohihasfoiaf


def get_todo_de_c_abnb():   
    import datetime
    import csv

    ### Este es el bloque dond se sencuentra tiulo, precio, camas rate etc
    try: 
        bloques = driver.find_elements(By.XPATH, "//div[@class='g1qv1ctd cb4nyux dir dir-ltr']")
        bloques = bloques[4:6]
        time.sleep(0.5)

        ### Este es e bloque dodne se enecuntra HALLAGOS o la IMAGEN de cada abnb esa cosa. Ahi haremos click
        ### para poder entrar a cada ABNB y sacar toda la info posible
        bloques_hallazgo = driver.find_elements(By.XPATH, "//div[@class='c14dgvke dir dir-ltr']")
        
        bloques_hallazgo = bloques_hallazgo[4:6]
        time.sleep(0.5)
        

        for x in bloques_hallazgo:   
            
            import datetime
            import csv

        
            #### declaramos las listas en las que se va a hacer el APPEND
            #### ---------------------------------------------------------------------------------------------
            
            global titulo_abnb

            global huespedes
            global banos
            global verificados 
            global habitacion 
            global cama 

            global limpieza_rate
            global comunicacion_rate 
            global llegada_rate
            global fiabilidad_rate 
            global ubicacion_rate 
            global precio_rate

            global reviews_lista

            global que_suerte

            global aircover 
            
            
            titulo_abnb = []
            huespedes = []
            banos = []
            verificados = []
            habitacion = []
            cama = []
            limpieza_rate = []
            comunicacion_rate = []
            llegada_rate = []
            fiabilidad_rate = []
            ubicacion_rate = []
            precio_rate = []
            reviews_lista=[]
            que_suerte = []
            aircover = []
            


            #### ---------------------------------------------------------------------------------------------
            
            
            time.sleep(2.6)
            x.click()
            time.sleep(1.8)

            print(driver.current_url)            
            time.sleep(0.6)

            driver.switch_to.window(driver.window_handles[1])
            time.sleep(2.2)
            
            print(driver.current_url)    
            
            time.sleep(2.5)

            try:
                cerrar_pop_up = driver.find_element(By.XPATH, "//div[@class='_1piuevz']")
                cerrar_pop_up.click()
                time.sleep(0.8)

            except:
                pass
            
            ## entonces para cada uno corremos las definicioes que scrapean las cosas de los ABNBs
            ######## ================================================================================= ###########
            get_importantes_abnb()
            time.sleep(0.8)

            
            
    
            date = datetime.date.today()
            
            data_ttl =     [date, titulo_abnb,huespedes, banos , verificados ,habitacion , cama , limpieza_rate , 
            comunicacion_rate , llegada_rate , fiabilidad_rate , ubicacion_rate , precio_rate , reviews_lista, 
            que_suerte , aircover]
            
            
            ### NTES DE AHCERLO REVISA SI ES SOBREESCRITO O RECIEN NUEVO
            with open('abnb_SSS_scrapper.csv', 'a+', newline='', encoding='UTF8') as f:
                writer = csv.writer(f)
                writer.writerow(data_ttl)
            
                time.sleep(6.8)
            
            ## regresamos a la pagina donde están todas las opciones de abnbs que tenemos
            driver.close()
            time.sleep(2.3)
            driver.switch_to.window(driver.window_handles[0])

    except:
        pass

In [22]:
get_todo_de_c_abnb()

https://www.airbnb.com.pe/s/Paris/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&query=Par%C3%ADs%2C%20Francia&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&date_picker_type=calendar&checkin=2023-04-19&checkout=2023-04-22&adults=1&source=structured_search_input_header&search_type=autocomplete_click
https://www.airbnb.com.pe/rooms/775895954593336404?adults=1&check_in=2023-04-19&check_out=2023-04-22&federated_search_id=6d64629f-67ae-42e3-a6a1-72be18e675b4&source_impression_id=p3_1677970827_wvh7cideaAyuxl4m


In [20]:
 driver.switch_to.window(driver.window_handles[0])

In [21]:
get_importantes_abnb()
limpieza_rate

['4.8']

In [26]:
import datetime
import csv

date = datetime.date.today()
data_ttl =     [date, titulo_abnb,huespedes, banos , verificados ,habitacion , cama , limpieza_rate , 
comunicacion_rate , llegada_rate , fiabilidad_rate , ubicacion_rate , precio_rate , reviews_lista, 
que_suerte , aircover]


### NTES DE AHCERLO REVISA SI ES SOBREESCRITO O RECIEN NUEVO
with open('abnb_SSS_scrapper.csv', 'a+', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data_ttl)

In [83]:
titulo_abnb = []

huespedes = []
banos = []
verificados = [] #### cro que este lo botamos
habitacion = []
cama = []

limpieza_rate = []
comunicacion_rate = []
llegada_rate = []
fiabilidad_rate = []
ubicacion_rate = []
precio_rate = []

reviews_lista=[]

que_suerte = []

aircover = []
    

def get_importantes_abnb():
    
    bloque_basicos = driver.find_element(By.XPATH, "//ol[@class='lgx66tx dir dir-ltr']")
    bloque_basicos = bloque_basicos.find_elements(By.XPATH, ".//li[@class='l7n4lsf dir dir-ltr']")
    time.sleep(0.6)
    
    """global titulo_abnb

    global huespedes
    global banos
    global verificados 
    global habitacion 
    global cama 

    global limpieza_rate
    global comunicacion_rate 
    global llegada_rate
    global fiabilidad_rate 
    global ubicacion_rate 
    global precio_rate

    global reviews_lista

    global que_suerte

    global aircover """
    time.sleep(1)

    try:
        
        huespedes_1 = bloque_basicos[0].text
        huespedes.append(huespedes_1)

        habitacion_1 = bloque_basicos[1].text
        habitacion.append(habitacion_1)

        cama_1 = bloque_basicos[2].text
        cama.append(cama_1)

        banos_1 = bloque_basicos[3].text
        banos.append(banos_1)
        
        time.sleep(1.6)
        
    except:
        pass

    try:
        
        verificado = driver.find_element(By.XPATH, "//div[@class='l1dfad8f dir dir-ltr']")
        verificado = verificado.text
        verificados.append(verificado)
        time.sleep(1.6)

    except:
        pass

    ## titulo
    try:
        bloques_title = driver.find_element(By.XPATH, "//h2[@class='_14i3z6h']")
        texto_title = bloques_title.text.lower()
        titulo_abnb.append(texto_title)
        time.sleep(1.6)
    except:
        pass


    ## sacnado rates
    try:
        rates_limpieza_eso = driver.find_element(By.XPATH, "//div[@class='ciubx2o dir dir-ltr']")       
        rates_limpieza_eso = rates_limpieza_eso.find_elements(By.XPATH, ".//span[@class='_4oybiu']") 

        try:
            limpieza_rate.append(rates_limpieza_eso[0].text)
            fiabilidad_rate.append(rates_limpieza_eso[1].text)
            comunicacion_rate.append(rates_limpieza_eso[2].text)
            ubicacion_rate.append(rates_limpieza_eso[3].text)
            llegada_rate.append(rates_limpieza_eso[4].text)
            precio_rate.append(rates_limpieza_eso[5].text)
            time.sleep(1.6)

        except:
            pass
    except:
        pass


    ##  sacando las reviews
    try:
        
        boton_revs = driver.find_element(By.XPATH, "//button[@data-testid='pdp-show-all-reviews-button']")
        boton_revs.click()

        bloque_pop = driver.find_element(By.XPATH, "//div[@class='_14vzertx']")

         ## scrolleamos para que carguen todas resenas
        scroll = 0

        while scroll < 3:
            time.sleep(0.7)
            popup_reviews = driver.find_elements(By.CSS_SELECTOR, 'div[role=dialog] div[data-review-id]')
            driver.execute_script('arguments[0].scrollIntoView({block: "center", behavior: "smooth"});', popup_reviews[-1])
            time.sleep(0.7)
            scroll+=1


        bloque_revs = driver.find_element(By.XPATH, "//div[@class='_lao3nn']")    
        bloque_revs = bloque_revs.find_elements(By.XPATH, "//div[@class='r1are2x1 dir dir-ltr']") 

        ## ya que hay coemntarios del ANFITRION, tneemos q evitarlos por lo tanto coegmeos el mini boloque (donde se
        ## encuentra los comentarios y de vez en cuando la respsta del anfitrion <3), y dentro del minibloque
        ## escogemos solo el primer coment [1], que sería el del usuario que alquiló ese abnb

        lista_revs = []

        for u in bloque_revs:
            revs_usuarios = u.find_element(By.XPATH, ".//span[@class='ll4r2nl dir dir-ltr'][1]")
            time.sleep(0.7)
            texto = revs_usuarios.text
            lista_revs.append(texto)
            reviews_lista.append(lista_revs)
            time.sleep(0.6)
        
        
        boton_cerrar = driver.find_element(By.XPATH, "//div[@class='_pa35zs']")
        boton_cerrar = boton_cerrar.find_element(By.XPATH, ".//button[@class='l1j9v1wn czcfm7x dir dir-ltr']")
        boton_cerrar.click()
        
    except:
        pass

    ## vemos si "EL ALOJAMIENTO SUELE ESTAR RESERVADO"
    try:
        suerte = driver.find_element(By.XPATH, "//h2[@class='_mtn6h9x']")
        suerte_text = suerte.text

        que_suerte.append(suerte_text)
        time.sleep(0.6)
    except:
        pass

    ## Caso tenga AIRCOVER
    try:
        aircover_tiene = driver.find_element(By.XPATH, "//img[@class='l1li2ovm dir dir-ltr']")
        aircover.append("si")
        time.sleep(0.6)
    except:
        pass

In [85]:
get_importantes_abnb()

In [84]:
 driver.switch_to.window(driver.window_handles[1])

In [86]:
verificados

[]

In [93]:
    titulo_abnb
    
    #huespedes 
   # banos
    #verificados 
    #habitacion 
#cama 
    
    #limpieza_rate 
    #comunicacion_rate 
    #llegada_rate
    #fiabilidad_rate 
    #ubicacion_rate 
    #precio_rate 
    
    #reviews_lista
    
    #que_suerte
   # 
    #aircover 

['habitación privada en vivienda rentada - anfitrión: suqin']

# te falt ahacer ersotototoottoto

identidad verif  /
promotor   /
idiomas ladra /
cancelacion /
Zonas de trabajos esas (3) webndas /
Cuando se registro el anfitrion  /
Costos HORMIGA  /
Servicios  //
LO QUE DEBES SBAER  
nombre del perro /
Dscto x estadia mensual /

In [59]:
driver.close()

In [60]:
import requests
from bs4 import BeautifulSoup
import time


#url2 = "https://www.airbnb.com.pe/rooms/38619376?adults=1&check_in=2023-04-08&check_out=2023-05-12&federated_search_id=577580e6-f39e-41a2-843d-0bafb9d566c4&source_impression_id=p3_1677295501_IWlq95rF2GWC6%2FzP"
url2 = "https://www.airbnb.com.pe/rooms/28713559?adults=1&check_in=2023-03-10&check_out=2023-03-18&federated_search_id=12409202-a6d8-4454-8d96-4de230b55ad7&source_impression_id=p3_1677383531_HYUq2EgrS3FrMov8"
r = requests.get(url2)

r.status_code

s = BeautifulSoup(r.text,'lxml')


from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

In [61]:
options = webdriver.ChromeOptions()
options.add_argument("--incognito")

driver = webdriver.Chrome(executable_path=r"F:\\Chrome driver\\chromedriver.exe", options = options)

driver.get(url2)

<ipython-input-61-1e3b6eee0013>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"F:\\Chrome driver\\chromedriver.exe", options = options)


In [72]:
idver = []
superanfitrion = []
Promotor_abnb = []

def get_things_del_anfi():
    try:
        ## Buscmas que es el huevas si es anfition, identidad verificada o si es promotor de abnb
        ## La vaina es que hay varios con esa clase (gracias airbnb programmers hsdlagranputa) y pueden ir de 0 a 4 webds con la misma clase
        ## entre ellos anfitrion, promotor, id_verificad, reseñas y quizá por ahi haya todavia alguna webda mas, ya se jode quien se lo chante
        ## entonces iniciamos un bucle para hacer un match entre los textos extraidos de cada "elemento" (solo sirven ENG y ESP)
        raa = driver.find_elements(By.XPATH, "//span[@class='l1dfad8f dir dir-ltr']")

        textos_identidad = ["identidad","verified"]
        super_huevon = ["superanfitrión", "superhost"]
        airbnb_promotor = ["airbnb.org", "promotor"]

        ## hacemos un match de cada "raa" que serían las vainas esas idver,super,promotor para ver si esos sapos feos
        for r in raa:
            r = (r.text).lower()    

            if any(ext in r for ext in textos_identidad):
                idver.append(r)
                time.sleep(0.4)

            elif any(ext in r for ext in super_huevon):
                superanfitrion.append(r)
                time.sleep(0.4)

            elif any(ext in r for ext in airbnb_promotor):
                Promotor_abnb.append(r) 
                time.sleep(0.4)

            else:
                pass
    except:
        pass

In [64]:
get_things_del_anfi()

In [67]:
idver
#superanfitrion 
#Promotor_abnb

['identidad verificada']

In [73]:
name_anfitiron = []
fecha_registro = []

def get_name_del_anfi():
    try:
        ## Buscmas que idioma ladra roof
        ## Como hay varios con esa clase, entonce sbuscamos el bloque y leugo dentro del bloque buscamos la clase.

        time.sleep(0.3)

        block = driver.find_element(By.XPATH, "//div[@class='tehcqxo dir dir-ltr']")
        name = block.find_element(By.XPATH, ".//h2[@class='hnwb2pb dir dir-ltr']")
        fecha_re = block.find_element(By.XPATH, ".//li[@class='l7n4lsf dir dir-ltr']")

        time.sleep(0.4)

        time.sleep(0.6)

        fecha_re = fecha_re.text
        fecha_re = fecha_re.lower()

        name = name.text
        name = name.lower()

        ##Split texto para que quede solo el name  "Anfitrion: Jorge"
        time.sleep(0.3)
        nemu = name.split(":")
        nemu = nemu[1]   ### Nos quedamos solo con el name de la vaina spltieada
        nemu = nemu[1:]  ### Quitamos el primer space
        time.sleep(0.3)
        name_anfitiron.append(nemu)

        ## Split texto de fehca inscripcion para que solo quede el año
        time.sleep(0.3)
        fecha_re = fecha_re.split(" ")
        fecha_registro.append(fecha_re[-1])
        time.sleep(0.3)

    except:
        pass

In [69]:
get_name_del_anfi()

In [72]:
name_anfitiron 
fecha_registro 

['2018']

In [94]:
zona_de_trabajo = []
acceso_sin_hr_restriccion = []
cancelacion_gratis = []

def get_cosas_abnb():
    try:
        ## Buscmas que idioma ladra roof
        ## Como hay varios con esa clase, entonce sbuscamos el bloque y leugo dentro del bloque buscamos la clase.

        bloques_minis = driver.find_elements(By.XPATH, "//div[@class='_1qsawv5']")
        time.sleep(0.6)

        zonas = ["zona","workspace"]
        accessso = ["acceso", "check-in"]
        gratuita_cance = ["gratuita", "cancellation"]

        for b in bloques_minis:
            b = b.text
            b = b.lower()

            time.sleep(0.4)

            if any(ext in b for ext in zonas):
                si = "si"
                zona_de_trabajo.append(si)
                time.sleep(0.4)

            elif any(ext in b for ext in accessso):
                si = "si"
                acceso_sin_hr_restriccion.append(si)
                time.sleep(0.4)

            elif any(ext in b for ext in gratuita_cance):
                ### Aca ya tienes que hacer data cleaning o ver pa que te serviria
                cancelacion_gratis.append(b)

                time.sleep(0.5)

            else:
                pass

    except:
        pass

In [98]:
cancelacion_gratis

['cancelación gratuita por 48 horas.']

In [95]:
get_cosas_abnb()

In [75]:
dscto_mensual = []
dscto_semanal = []
tarifa_service = []
tarifa_cleaning = []

precio_noche_mes_week = []
precio_total = []
precio_original = []

q_resenas = []
puntaje_place = []

def get_prices_dscts_abnb():
    try:    

        ### Primero encontramos el precio x mes noche dia:
        bloque_precio_night = driver.find_elements(By.XPATH, "//span[@class='_14y1gc']")
        bloque_precio_night = bloque_precio_night[1].find_element(By.XPATH, ".//span[@class='a8jt5op dir dir-ltr']")
        texto = bloque_precio_night.text

        try:
            z = texto.split()
            z_0 = z[0]
            precio_noche_mes_week.append(z_0)

            if len(z) > 4:
                z_original = z[-1]
                precio_original.append(z_original)

            else:
                pass

        except:
            precio_noche_mes_week.append(texto)


        ### Encontramos el precio total       
        bloque_precio_completo = driver.find_elements(By.XPATH, ".//span[@class='_j1kt73']")
        texto_p_c = bloque_precio_completo[1].text

        try:
            z = texto_p_c.split()
            z = z[0]
            precio_total.append(z)
        except:
            precio_total.append(texto_p_c)

        ### ------------------------------------------------------------------------------------------------------------------


        ### A buscar la cantidad de reseñas y rate que tiene
        bloque_precio_night = driver.find_element(By.XPATH, "//span[@class='_1y9gwt75']")

        puntaje = bloque_precio_night.find_element(By.XPATH, ".//span[@class='_12si43g']")
        putnaje_texto = puntaje.text
        p_t = putnaje_texto.split()
        p_t = float(p_t[0])

        q_de_resenas = bloque_precio_night.find_element(By.XPATH, ".//span[@class='_1jlwy4xq']")
        q_resenas_texto = q_de_resenas.text
        q_r_t = q_resenas_texto.split()
        q_r_t = int(q_r_t[0])

        puntaje_place.append(p_t)
        q_resenas.append(q_r_t)
    except:
        pass

        # --------------------------------------------------------------------------------------------
    try:
        ## En el bloque grande vamo a buscar los dsctos y tarifas
        bloque_grande = driver.find_elements(By.XPATH, "//div[@class='_1fpuhdl']")

        time.sleep(0.4)

        limpia = ["limpieza","cleaning"]
        servicio = ["servicio", "service"]
        dscto = ["descuento", "discount"]

        coma=[",","S","$","U","-","/"]

        for bloque in bloque_grande:
            bloques_minis = bloque.find_elements(By.XPATH, ".//div[@class='_m6lwl6']")
            bloques_minis_dscto = bloque.find_elements(By.XPATH, ".//span[@class='_18x3iiu']")

            time.sleep(0.4)

            for b,c in zip(bloques_minis,bloques_minis_dscto):
                be = b.text
                be = be.lower()       ### be es del blqoue limpieza,service, s ehace mas facil con ese bloque:minis xq no hay "/n"

                time.sleep(0.4)

                ce = c.text         #### ce es del bloque de DESCUENTO xq los otros dos no son iguales a este
                ce = ce.lower()

                time.sleep(0.4)

                if any(ext in be for ext in servicio):
                    si = bloque.find_element(By.XPATH, ".//span[@class='_1k4xcdh']")
                    si = si.text

                    time.sleep(0.4)

                    res=[j for i,j in enumerate(si) if j not in coma]
                    price = "".join(res)
                    price = int(price)

                    tarifa_service.append(price)

                    time.sleep(0.5)

                elif any(ext in be for ext in limpia):
                    si = bloque.find_element(By.XPATH, ".//span[@class='_1k4xcdh']")
                    si = si.text

                    time.sleep(0.4)

                    res=[j for i,j in enumerate(si) if j not in coma]
                    price = "".join(res)
                    price = int(price)

                    time.sleep(0.4)

                    tarifa_cleaning.append(price)

                    time.sleep(0.3)

                ## ya que hay descuentos semanasl, mensuales ttenemos que dividirlos en dos columnas diferentes
                ## primero "fitlramos" si el webelement texto tiene la palabra DESCUENTO, y leugo sacamos si es sem o mens
                elif any(ext in ce for ext in dscto):
                    dscto_mes = ["monthly", "mensual"]
                    dscto_sem = ["weekly", "semanal"]

                    if any(ext in ce for ext in dscto_mes):

                        si = bloque.find_element(By.XPATH, ".//span[@class='_1rc8xn5']")
                        si = si.text

                        time.sleep(0.4)
                        ### ya que tiene simbolo de $ S/. etc y negativos (-) etc, tneemos que hace run cleaning previo para eso p
                        ### 
                        res=[j for i,j in enumerate(si) if j not in coma]
                        price = "".join(res)
                        price = int(price)

                        dscto_mensual.append(price)

                        time.sleep(0.5)

                    elif any(ext in ce for ext in dscto_sem):

                        si = bloque.find_element(By.XPATH, ".//span[@class='_1rc8xn5']")
                        si = si.text

                        time.sleep(0.4)
                        ### ya que tiene simbolo de $ S/. etc y negativos (-) etc, tneemos que hace run cleaning previo para eso p
                        ### 
                        res=[j for i,j in enumerate(si) if j not in coma]
                        price = "".join(res)
                        price = int(price)

                        dscto_semanal.append(price)

                        time.sleep(0.5)

                    else:
                        pass

                else:
                    pass

    except:
        pass

In [118]:
get_prices_dscts_abnb()

In [128]:
dscto_mensual
dscto_semanal
#tarifa_service
#tarifa_cleaning

#precio_noche_mes_week
#precio_total
#precio_original

q_resenas
puntaje_place

[4.66]

In [76]:
idiomas = []
resposne_rate = []
response_time = []

def communication_anfit():
    try:
        ## Buscmas que idioma ladra roof, el indice de rpsta y el tiempo de rspsta
        ## Como hay varios con esas 3 clases, entonce sbuscamos el bloque y leugo dentro del bloque buscamos las pre-clases 
        ## que serviran para hacer un match de subitulo con el contendio I.E: "Tiempo de respuesta: en menos de una hora".
        ## con ese match hecho entonces buscamos la clase donde se encuentra el contenido par aextraerlo, solo si se encuentra ese match.

        bloque = driver.find_element(By.XPATH, "//div[@data-section-id='HOST_PROFILE_DEFAULT']")
        raa = bloque.find_elements(By.XPATH, ".//li[@class='f19phm7j dir dir-ltr']")

        lenguaje = ["idiomas","languages"]
        response_rate = ["índice", "rate"]
        resp_time = ["time", "tiempo"]

        time.sleep(1.3)

        ## hacemos un match de cada "raa"
        for r in raa:
            re = (r.text).lower()    

            if any(ext in re for ext in lenguaje):
                ## caso exista el match entonces dentro de ese RAA que sería "r" se busca el contendio
                languages = r.find_element(By.XPATH, ".//span[@class='ll4r2nl dir dir-ltr']")
                languages = languages.text
                languages = languages.lower()
                time.sleep(0.6)

                coma = [","]
                ## Ya que algunos solo hablan un idioma tenemos que bsucar una forma de ahcer el append sin splitear:    
                if any(ext in languages for ext in coma):
                ##spliteamos los idiomas si se peude para tenerlo en una lista y simplificar el proceso de Data Explor o creacion d Algorit
                    l = languages.split(",")
                    idiomas.append(l)
                    time.sleep(0.6)
                else:
                    idiomas.append(languages)
                    time.sleep(0.6)

            elif any(ext in re for ext in response_rate):
                languages = r.find_element(By.XPATH, ".//span[@class='ll4r2nl dir dir-ltr']")
                languages = languages.text
                languages = languages.lower()
                resposne_rate.append(languages)
                time.sleep(0.6)

            elif any(ext in re for ext in resp_time):
                languages = r.find_element(By.XPATH, ".//span[@class='ll4r2nl dir dir-ltr']")
                languages = languages.text
                languages = languages.lower()
                response_time.append(languages)
                time.sleep(0.6)

            else:
                pass

    except:
        pass

In [91]:
driver.switch_to.window(driver.window_handles[2])

In [92]:
communication_anfit()

In [95]:
idiomas
#resposne_rate
#response_time 

[['english', ' français', ' português', ' español'],
 ['中文 (简体)', ' english', ' français']]

In [77]:
servicios_solos = []

bano_servicios = []
dormitorio_lavadero = []
entretenimiento = []

familia = []
calefaccion_refrigeracion = []
seguridad_hogar = []

internet_oficina = []
utensilios_vajilla = []
estacionamiento_instalaciones = []

ubicacion_servicios = []
exterior_servicios = []
servicios_servicios = []



no_incluidos = []

def get_servicios_abnb():
    try:
        ## Buscmas que idioma ladra roof, el indice de rpsta y el tiempo de rspsta
        ## Como hay varios con esas 3 clases, entonce sbuscamos el bloque y leugo dentro del bloque buscamos las pre-clases 
        ## que serviran para hacer un match de subitulo con el contendio I.E: "Tiempo de respuesta: en menos de una hora".
        ## con ese match hecho entonces buscamos la clase donde se encuentra el contenido par aextraerlo, solo si se encuentra ese match.

        boton_services = driver.find_element(By.XPATH, "//button[@class='l1j9v1wn b65jmrv v7aged4 dir dir-ltr'][1]")
        boton_services.click()

        time.sleep(1.1)
        #me posiciono en el pop up
        bloque_pop = driver.find_element(By.XPATH, "//div[@class='_1seuw5go']")
        ## COJO LOS H3 PARA PODER AHCE RUN BUCLE PA CADA UNO
        ## cada h3 es un tipo_servicio que se brinda
        ## y en cada tipo_servicio se da varios elementos IE cocina: Cocina, paltos, ollas, cubeirtos, extractor
        los_h3 = bloque_pop.find_elements(By.XPATH, ".//h3")

        ## Ahora necesitamos de donde sacar los servicios que incluye cada tipo:
        division = bloque_pop.find_elements(By.XPATH, ".//div[@class='_11jhslp']")

        time.sleep(0.6)

        ### las listas par ahacer los matches:
        bano_servicios_matc = ["baño", "bathroom"]
        dormitorio_lavadero_matc = ["dormitorio", "bedroom"]
        entretenimiento_matc = ["entretenimiento","entertainment"]

        familia_matc = ["familia","family"]
        calefaccion_refrigeracion_matc = ["calefacción","heating"]
        seguridad_hogar_matc = ["seguridad", "safety"]

        internet_oficina_matc = ["internet"]
        utensilios_vajilla_matc = ["kitchen","vajilla"]
        estacionamiento_instalaciones_matc = ["estacionamiento","parking"]

        ubicacion_servicios_matc = ["ubicación","location"]
        exterior_servicios_matc = ["exterior","outdoor"]
        servicios_servicios_matc = ["servicios","services"]

        no_incluidos_matc = ["incluido","included"]

        time.sleep(0.3)
        ### las listas par ahacer los matches DONE


        ## hacemos un for con los h3 y la division para hacer el match y segun eso sacar los servicios
        for h3_, divi in zip(los_h3,division):

            ## para hacer el match scamaos el texto
            h3_tipo_servicio = h3_.text.lower()

            ## dentro de la division buscamos los elementos (servicioss)
            servicio = divi.find_elements(By.XPATH, ".//div[@class='_gfomxi']")

            time.sleep(0.6)

    ## hacemos un match -----------------------------------------------------------------------------------------------
            if any(ext in h3_tipo_servicio for ext in bano_servicios_matc):
                ## caso exista el match entonces sacamos LOS TEXTOS de servicio y lo mandamos a la lista
                for servi in servicio:
                    texto = servi.text.lower()           
                    bano_servicios.append(texto)
                    time.sleep(0.6)

            elif any(ext in h3_tipo_servicio for ext in dormitorio_lavadero_matc):

                for servi in servicio:
                    texto = servi.text.lower()           
                    dormitorio_lavadero.append(texto)
                    time.sleep(0.6)
    # --------------------------------------------------------------------------------------------
            elif any(ext in h3_tipo_servicio for ext in entretenimiento_matc):

                for servi in servicio:
                    texto = servi.text.lower()           
                    entretenimiento.append(texto)     
                    time.sleep(0.6)

            elif any(ext in h3_tipo_servicio for ext in familia_matc):

                for servi in servicio:
                    texto = servi.text.lower()           
                    familia.append(texto)
                    time.sleep(0.6)

            elif any(ext in h3_tipo_servicio for ext in calefaccion_refrigeracion_matc):

                for servi in servicio:
                    texto = servi.text.lower()           
                    calefaccion_refrigeracion.append(texto)
                    time.sleep(0.6)
     # --------------------------------------------------------------------------------------------           
            elif any(ext in h3_tipo_servicio for ext in seguridad_hogar_matc):

                for servi in servicio:
                    texto = servi.text.lower()           
                    seguridad_hogar.append(texto)
                    time.sleep(0.6)

            elif any(ext in h3_tipo_servicio for ext in internet_oficina_matc):

                for servi in servicio:
                    texto = servi.text.lower()           
                    internet_oficina.append(texto)
                    time.sleep(0.6)

            elif any(ext in h3_tipo_servicio for ext in utensilios_vajilla_matc):

                for servi in servicio:
                    texto = servi.text.lower()           
                    utensilios_vajilla.append(texto)
                    time.sleep(0.6)
     # --------------------------------------------------------------------------------------------           
    ## 
            elif any(ext in h3_tipo_servicio for ext in estacionamiento_instalaciones_matc):

                for servi in servicio:
                    texto = servi.text.lower()           
                    estacionamiento_instalaciones.append(texto)
                    time.sleep(0.6)

            elif any(ext in h3_tipo_servicio for ext in ubicacion_servicios_matc):

                for servi in servicio:
                    texto = servi.text.lower()           
                    ubicacion_servicios.append(texto)
                    time.sleep(0.6)

            elif any(ext in h3_tipo_servicio for ext in exterior_servicios_matc):

                for servi in servicio:
                    texto = servi.text.lower()           
                    exterior_servicios.append(texto)
                    time.sleep(0.6)

            elif any(ext in h3_tipo_servicio for ext in servicios_servicios_matc):

                for servi in servicio:
                    texto = servi.text.lower()           
                    servicios_servicios.append(texto)
                    time.sleep(0.6)

            elif any(ext in h3_tipo_servicio for ext in no_incluidos_matc):

                for servi in servicio:
                    texto = servi.text.lower()           
                    no_incluidos.append(texto)
                    time.sleep(0.6)        

            else:
                pass
            
        bloq_cerrar = driver.find_element(By.XPATH, "//div[@class='_11f1y59w']")
        boton_close = bloq_cerrar.find_element(By.XPATH, ".//span[@class='i1h5tsj6 dir dir-ltr']")
        time.sleep(0.6)
        boton_close.click()

    except:

        bloque_servicios = driver.find_element(By.XPATH, "//div[@class='_1byskwn']")
        raa = bloque_servicios.find_elements(By.XPATH, ".//div[@class='iikjzje i10xc1ab dir dir-ltr']")

        time.sleep(0.6)

        for x in raa:
            texto = x.text
            texto = texto.lower()     
            servicios_solos.append(texto)

            time.sleep(0.8)        


In [160]:
get_servicios_abnb()

In [163]:
servicios_solos 

bano_servicios 
#dormitorio_lavadero 
#entretenimiento 
#familia 
#calefaccion_refrigeracion
#seguridad_hogar

#internet_oficina 
#utensilios_vajilla 
#estacionamiento_instalaciones

#ubicacion_servicios 
#exterior_servicios 
#servicios_servicios

#no_incluidos

['secadora de pelo', 'agua caliente']

In [78]:
reglas_casa = []
seguridad_y_propiedad = []
politica_cancelacion = []

def politicas_abnb():
    try:
        ## Buscmas que es el huevas si es anfition, identidad verificada o si es promotor de abnb
        ## La vaina es que hay varios con esa clase (gracias airbnb programmers hsdlagranputa) y pueden ir de 0 a 4 webds con la misma clase
        ## entre ellos anfitrion, promotor, id_verificad, reseñas y quizá por ahi haya todavia alguna webda mas, ya se jode quien se lo chante
        ## entonces iniciamos un bucle para hacer un match entre los textos extraidos de cada "elemento" (solo sirven ENG y ESP)
        bloque_grande = driver.find_element(By.XPATH, "//div[@data-section-id='POLICIES_DEFAULT']")
        buttons = bloque_grande.find_elements(By.XPATH, ".//span[@class='cn5lml1 dir dir-ltr']")
        time.sleep(0.3)


        buton_reglas = buttons[0]
        buton_seguridad = buttons[1]
        buton_politc = buttons[2]
        time.sleep(0.3)

                         ##--------------------------------------------------------------------------------

        ##primer boton
        buton_reglas.click()

        time.sleep(1)

        ## Para el priemr popup
        bloque = driver.find_element(By.XPATH, "//div[@class='_17itzz4']")
        bloque_peque = bloque.find_elements(By.XPATH, ".//div[@class='ce5nonf dir dir-ltr']")

        for elemento in bloque_peque:
            texto = elemento.text.lower()
            reglas_casa.append(texto) 

        salir = driver.find_element(By.XPATH, "//div[@class='_pa35zs']")
        salir.click()
        time.sleep(0.6)

                 ##--------------------------------------------------------------------------------

        ## sgundo boton

        buton_seguridad.click()

        time.sleep(1)

        ## Para el segundo  popup
        bloque = driver.find_element(By.XPATH, "//div[@class='_17itzz4']")
        bloque_peque = bloque.find_elements(By.XPATH, ".//div[@class='ce5nonf dir dir-ltr']")

        for elemento in bloque_peque:
            texto = elemento.text.lower()
            seguridad_y_propiedad.append(texto)

        salir = driver.find_element(By.XPATH, "//div[@class='_pa35zs']")
        salir.click()
        time.sleep(0.6)


                ##--------------------------------------------------------------------------------

        ## tercer boton
        buton_politc.click()

        time.sleep(1.5)

        ## Para el tercer popup

        bloque_peque = driver.find_elements(By.XPATH, "//div[@class='_dj903s']")


        for elemento in bloque_peque:

            texto = elemento.text.lower()

            politica_cancelacion.append(texto)
            time.sleep(1)

        salir = driver.find_element(By.XPATH, "//div[@class='_pa35zs']")
        salir.click()
        time.sleep(0.6)

    except:
        raa = driver.find_element(By.XPATH, "//div[@class='c1e17v3g dir dir-ltr']")
        divs = raa.find_elements(By.XPATH, ".//div[@class='is50c2g dir dir-ltr']")
        time.sleep(0.3)

        for div in divs:
            texto = div.text.lower()
            reglas_casa.append(texto)
            time.sleep(0.7)

            ## -----------------------------------------------------------------------------

        raa_2 = driver.find_element(By.XPATH, "//div[@class='c1e17v3g dir dir-ltr'][1]")
        divs_2 = raa_2.find_elements(By.XPATH, ".//div[@class='is50c2g dir dir-ltr']")
        time.sleep(0.3)

        for div in divs_2:
            texto = div.text.lower()
            seguridad_y_propiedad.append(texto)
            time.sleep(0.7)

            ## -----------------------------------------------------------------------------

        raa_3 = driver.find_element(By.XPATH, "//div[@class='bs4u5d4 v6231dh dir dir-ltr'][3]")
        divs_3 = raa_3.find_elements(By.XPATH, ".//div[@class='h4lj7td dir dir-ltr']")
        time.sleep(0.3)

        for div in divs_3:
            texto = div.text.lower()
            politica_cancelacion.append(texto)
            time.sleep(0.7)

In [165]:
politicas_abnb()

In [168]:
politica_cancelacion 
#reglas_casa
seguridad_y_propiedad 

['dispositivos de seguridad\nalarma de monóxido de carbono no informada\nel anfitrión no indicó que la propiedad contara con alarma de monóxido de carbono. te sugerimos que lleves un detector portátil en el viaje.\ndetector de humo no informado\nel anfitrión no indicó que hubiera un detector de humo en la propiedad. te sugerimos que lleves un detector portátil en el viaje.',
 'información de la propiedad\nhay armas en la propiedad',
 'el horario de check-in es de las 17:00 a las 22:00',
 'la salida es antes de las 11:00',
 '2 huéspedes como máximo']

In [169]:

import datetime

today = datetime.date.today()

## Guardamos en un DF, puedes crear uno antes con un HEADER como el declarado abajo.
import csv     

header = ['date','titulo', 'huespedes','banos','verificado','habitaciones','camas','limpieza_rate','comunicacion_rate',
          'llegada_rate','fiabilidad_rate','ubicacion_rate','precio_rate','reviews_lista','que_suerte','aircover','idver',
          'superanfitrion','Promotor_abnb','name_anfitiron','fecha_registro',
          'zona_de_trabajo','acceso_sin_hr_restriccion','cancelacion_gratis','dscto_mensual',
          'dscto_semanal','tarifa_service','tarifa_cleaning','precio_noche_mes_week','precio_total',
          'precio_original','q_resenas','puntaje_place','idiomas','resposne_rate','response_time',
          'servicios_solos','bano_servicios','dormitorio_lavadero','entretenimiento','familia',"calefaccion_refrigeracion"
          'seguridad_hogar','internet_oficina','utensilios_vajilla','estacionamiento_instalaciones','ubicacion_servicios',
          'exterior_servicios','servicios_servicios','no_incluidos','reglas_casa','seguridad_y_propiedad',
          'politica_cancelacion']

data =     [date, titulo_abnb,huespedes, banos , verificados ,habitacion , cama , limpieza_rate , 
comunicacion_rate , llegada_rate , fiabilidad_rate , ubicacion_rate , precio_rate , reviews_lista, 
que_suerte , aircover , idver , superanfitrion , Promotor_abnb , name_anfitiron , 
fecha_registro , zona_de_trabajo ,acceso_sin_hr_restriccion , cancelacion_gratis ,dscto_mensual ,
dscto_semanal , tarifa_service ,tarifa_cleaning , precio_noche_mes_week , precio_total ,
precio_original , q_resenas , puntaje_place , idiomas , resposne_rate , response_time ,            
servicios_solos , bano_servicios , dormitorio_lavadero , entretenimiento , familia , calefaccion_refrigeracion ,
seguridad_hogar , internet_oficina , utensilios_vajilla , estacionamiento_instalaciones , ubicacion_servicios ,
exterior_servicios , servicios_servicios , no_incluidos , reglas_casa , seguridad_y_propiedad ,
politica_cancelacion ]

with open('abnb.csv', 'a+', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data)


NameError: name 'titulo_abnb' is not defined

In [170]:
import pandas as pd
daf = pd.read_csv("abnb.csv")

# Creo que una vez creas la lista, si no hay records, aparece como NULL cuando haces el append al DF, haz la prueba con justo este de arriba

In [97]:
daf

,['promotor de airbnb.org'],['superanfitrión'],['identidad verificada'],2023-02-25
0,['promotor de airbnb.org'],['superanfitrión'],['identidad verificada'],2023-02-25
1,[],[],['identidad verificada'],2023-02-25


In [68]:
url2 = "https://www.airbnb.com.pe/rooms/38619376?adults=1&check_in=2023-04-08&check_out=2023-05-12&federated_search_id=577580e6-f39e-41a2-843d-0bafb9d566c4&source_impression_id=p3_1677295501_IWlq95rF2GWC6%2FzP"

In [55]:
politica_cancelacion

['3 mar.\n17:00\nreembolso parcial: recibirás el 50 % del monto de todas las noches. no se te reembolsará la tarifa por servicio.',
 '10 mar.\n17:00\n(check-in)\nsin reembolso']

In [56]:
reglas_casa

['quién puede quedarse en el alojamiento\n2 huéspedes como máximo\nno se admiten mascotas',
 'qué está permitido\nel horario de check-in es de las 17:00 a las 22:00\nla salida es antes de las 11:00\nacceso sin restricción de horario con caja de seguridad con llaves\nestá prohibido hacer fiestas o eventos\nprohibido fumar']

In [57]:
seguridad_y_propiedad

['dispositivos de seguridad\nalarma de monóxido de carbono no informada\nel anfitrión no indicó que la propiedad contara con alarma de monóxido de carbono. te sugerimos que lleves un detector portátil en el viaje.\ndetector de humo no informado\nel anfitrión no indicó que hubiera un detector de humo en la propiedad. te sugerimos que lleves un detector portátil en el viaje.',
 'información de la propiedad\nhay armas en la propiedad']

In [58]:
driver.close()

In [6]:
header = ['date','titulo', 'huespedes','banos','verificado','habitaciones','camas','limpieza_rate','comunicacion_rate',
          'llegada_rate','fiabilidad_rate','ubicacion_rate','precio_rate','reviews_lista','que_suerte','aircover','idver',
          'superanfitrion','Promotor_abnb','name_anfitiron','fecha_registro',
          'zona_de_trabajo','acceso_sin_hr_restriccion','cancelacion_gratis','dscto_mensual',
          'dscto_semanal','tarifa_service','tarifa_cleaning','precio_noche_mes_week','precio_total',
          'precio_original','q_resenas','puntaje_place','idiomas','resposne_rate','response_time',
          'servicios_solos','bano_servicios','dormitorio_lavadero','entretenimiento','familia',"calefaccion_refrigeracion"
          'seguridad_hogar','internet_oficina','utensilios_vajilla','estacionamiento_instalaciones','ubicacion_servicios',
          'exterior_servicios','servicios_servicios','no_incluidos','reglas_casa','seguridad_y_propiedad',
          'politica_cancelacion']

data =     ["titulo_abnb","huespedes", "banos" , "verificados" ,"habitacion" , "cama" , "limpieza_rate" , 
"comunicacion_rate" , "llegada_rate" , "fiabilidad_rate" , "ubicacion_rate" , "precio_rate" , "reviews_lista", 
"que_suerte" , "aircover" , "idver" , "superanfitrion" , "Promotor_abnb" , "name_anfitiron" , 
"fecha_registro" , "zona_de_trabajo" ,"acceso_sin_hr_restriccion" , "cancelacion_gratis" ,"dscto_mensual" ,
"dscto_semanal" , "tarifa_service" ,"tarifa_cleaning" , "precio_noche_mes_week" , "precio_total" ,
"precio_original" , "q_resenas" , "puntaje_place" , "idiomas" , "resposne_rate" , "response_time" ,            
"servicios_solos" , "bano_servicios" , "dormitorio_lavadero" , "entretenimiento" , "familia" , "calefaccion_refrigeracion" ,
"seguridad_hogar" , "internet_oficina" , "utensilios_vajilla" , "estacionamiento_instalaciones" , "ubicacion_servicios" ,
"exterior_servicios" , "servicios_servicios" , "no_incluidos" , "reglas_casa" , "seguridad_y_propiedad" ,
"politica_cancelacion" ]

In [7]:
len(header)

52

In [9]:
len(data)

52

In [34]:
bloque_precio_completo = driver.find_elements(By.XPATH, "//span[@class='_14y1gc']")
bloque_precio_completo = bloque_precio_completo[1].find_elements(By.XPATH, ".//span[@class='a8jt5op dir dir-ltr']")

In [36]:
z = bloque_precio_completo[0].text

In [43]:
bloque_precio_completo = driver.find_elements(By.XPATH, ".//span[@class='_j1kt73']")
texto_p_c = bloque_precio_completo[1].text

In [44]:
bloque_precio_completo[1].text


'S/714'

In [31]:
bloque_precio_completo[1].text

'S/4,159'

In [38]:
z.split()

['S/3,192', 'por', 'mes,', 'precio', 'original', 'S/4,560']

In [53]:
    bloque_precio_night = driver.find_element(By.XPATH, "//span[@class='_1y9gwt75']")
    
    puntaje = bloque_precio_night.find_element(By.XPATH, ".//span[@class='_12si43g']")
    putnaje_texto = puntaje.text
    p_t = putnaje_texto.split()
    p_t = float(p_t[0])
    
    q_de_resenas = bloque_precio_night.find_element(By.XPATH, ".//span[@class='_1jlwy4xq']")
    q_resenas_texto = q_de_resenas.text
    q_r_t = q_resenas_texto.split()
    q_r_t = int(q_r_t[0])

In [54]:
p_t

4.24

In [56]:
bloques = driver.find_elements(By.XPATH, "//div[@class='g1qv1ctd cb4nyux dir dir-ltr']")

In [57]:
bloques

[]

In [3]:
    bloques_title = driver.find_element(By.XPATH, "//h2[@class='_14i3z6h']")
    texto = bloques_title.text.lower()

In [4]:
texto

'habitación privada en vivienda rentada - anfitrión: gomes de oliveira'

In [210]:
bloque_basicos = driver.find_element(By.XPATH, "//ol[@class='lgx66tx dir dir-ltr']")
bloque_basicos = bloque_basicos.find_elements(By.XPATH, "//li[@class='l7n4lsf dir dir-ltr']")

In [212]:
bloque_basicos[0].text

''

In [ ]:
header = ['date','titulo', 'huespedes','banos','verificado','habitaciones','camas','limpieza_rate','comunicacion_rate',
          'llegada_rate','fiabilidad_rate','ubicacion_rate','precio_rate','reviews_lista','que_suerte','aircover','idver',
          'superanfitrion','Promotor_abnb','name_anfitiron','fecha_registro',
          'zona_de_trabajo','acceso_sin_hr_restriccion','cancelacion_gratis','dscto_mensual',
          'dscto_semanal','tarifa_service','tarifa_cleaning','precio_noche_mes_week','precio_total',
          'precio_original','q_resenas','puntaje_place','idiomas','resposne_rate','response_time',
          'servicios_solos','bano_servicios','dormitorio_lavadero','entretenimiento','familia',"calefaccion_refrigeracion"
          'seguridad_hogar','internet_oficina','utensilios_vajilla','estacionamiento_instalaciones','ubicacion_servicios',
          'exterior_servicios','servicios_servicios','no_incluidos','reglas_casa','seguridad_y_propiedad',
          'politica_cancelacion']

data =     [date, titulo_abnb,huespedes, banos , verificados ,habitacion , cama , limpieza_rate , 
comunicacion_rate , llegada_rate , fiabilidad_rate , ubicacion_rate , precio_rate , reviews_lista, 
que_suerte , aircover , idver , superanfitrion , Promotor_abnb , name_anfitiron , 
fecha_registro , zona_de_trabajo ,acceso_sin_hr_restriccion , cancelacion_gratis ,dscto_mensual ,
dscto_semanal , tarifa_service ,tarifa_cleaning , precio_noche_mes_week , precio_total ,
precio_original , q_resenas , puntaje_place , idiomas , resposne_rate , response_time ,            
servicios_solos , bano_servicios , dormitorio_lavadero , entretenimiento , familia , calefaccion_refrigeracion ,
seguridad_hogar , internet_oficina , utensilios_vajilla , estacionamiento_instalaciones , ubicacion_servicios ,
exterior_servicios , servicios_servicios , no_incluidos , reglas_casa , seguridad_y_propiedad ,
politica_cancelacion ]

In [70]:
header = ['date','titulo', 'huespedes','banos','verificado','habitaciones','camas','limpieza_rate','comunicacion_rate',
          'llegada_rate','fiabilidad_rate','ubicacion_rate','precio_rate','reviews_lista','que_suerte','aircover','idver',
          'superanfitrion','Promotor_abnb','name_anfitiron','fecha_registro',
          'zona_de_trabajo','acceso_sin_hr_restriccion','cancelacion_gratis','dscto_mensual',
          'dscto_semanal','tarifa_service','tarifa_cleaning','precio_noche_mes_week','precio_total',
          'precio_original','q_resenas','puntaje_place','idiomas','resposne_rate','response_time',
          'servicios_solos','bano_servicios','dormitorio_lavadero','entretenimiento','familia',"calefaccion_refrigeracion",
          'seguridad_hogar','internet_oficina','utensilios_vajilla','estacionamiento_instalaciones','ubicacion_servicios',
          'exterior_servicios','servicios_servicios','no_incluidos','reglas_casa','seguridad_y_propiedad',
          'politica_cancelacion']

In [71]:
len(header)

52

In [84]:
driver.close()

In [72]:
sa = ["date", "titulo_abnb","huespedes", "banos" , "verificados" ,"habitacion" , "cama" , "limpieza_rate" , 
            "comunicacion_rate" , 
      "llegada_rate" , "fiabilidad_rate" , "ubicacion_rate" , "precio_rate" , "reviews_lista", 
            "que_suerte" , "aircover" , "idver" , 
      "superanfitrion" , "Promotor_abnb" , "name_anfitiron" ], 
            "fecha_registro" , "zona_de_trabajo" ,"acceso_sin_hr_restriccion" , "cancelacion_gratis" ,"dscto_mensual" ,
            "dscto_semanal" , "tarifa_service" ,"tarifa_cleaning" , "precio_noche_mes_week" , "precio_total" ,
            "precio_original" , "q_resenas" , "puntaje_place" , "idiomas" , "resposne_rate" , "response_time" ,            
            "servicios_solos" , "bano_servicios" , "dormitorio_lavadero" , "entretenimiento" , "familia" , "calefaccion_refrigeracion" ,
            "seguridad_hogar" , "internet_oficina" , "utensilios_vajilla" , "estacionamiento_instalaciones" , "ubicacion_servicios" ,
            "exterior_servicios" , "servicios_servicios" , "no_incluidos" , "reglas_casa" , "seguridad_y_propiedad" ,
            "politica_cancelacion" ]

In [90]:
sa = ["resposne_rate" , "response_time" ,            
            "servicios_solos" , "bano_servicios" ,
      "dormitorio_lavadero" , "entretenimiento" , 
      "familia" , "calefaccion_refrigeracion" ,
            "seguridad_hogar" , "internet_oficina" ,
      "utensilios_vajilla" , "estacionamiento_instalaciones" ,
      "ubicacion_servicios" ,    "exterior_servicios" ,
      "servicios_servicios" , "no_incluidos" , 
      "reglas_casa" , "seguridad_y_propiedad" ,
            "politica_cancelacion" ]
len(sa)

19

In [92]:
header = ['resposne_rate','response_time',
          'servicios_solos','bano_servicios',
          'dormitorio_lavadero','entretenimiento',
          'familia','calefaccion_refrigeracion',
          'seguridad_hogar','internet_oficina',
          'utensilios_vajilla','estacionamiento_instalaciones',
          'ubicacion_servicios',    'exterior_servicios',
          'servicios_servicios','no_incluidos',
          'reglas_casa','seguridad_y_propiedad',
          'politica_cancelacion']
len(header)

19